In [9]:
# Imports
import os

from dotenv import load_dotenv

## LLMSherpa
from llmsherpa.readers import LayoutPDFReader

## LlamaIndex
from llama_index.core import VectorStoreIndex, Document, StorageContext
from llama_index.vector_stores.duckdb import DuckDBVectorStore
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingMode, OpenAIEmbeddingModelType
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding

## Traceloop
from traceloop.sdk import Traceloop

## LangChain
from langchain_community.document_loaders import UnstructuredPDFLoader

In [6]:
# os.environ["TRACELOOP_API_KEY"] = "your_traceloop_api_key"
# os.environ["GOOGLE_API_KEY"] = "your_google_api_key"
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
# os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_api_key"

# Loads .env file with the above API keys
load_dotenv(override=True)

True

In [7]:
# Enable OpenTelemetry auto-instrumentation using Traceloop's OpenLLMetry
Traceloop.init(disable_batch=True, api_key=os.environ["TRACELOOP_API_KEY"])

Traceloop syncing configuration and prompts
Traceloop exporting traces to https://api.traceloop.com authenticating with bearer token



In [8]:
# Split PDF into "context-aware" chunks, using LayoutPDFReader from llmsherpa API running locally.

llmsherpa_api_url = "http://localhost:5010/api/parseDocument?renderFormat=all&applyOcr=yes&useNewIndentParser=yes"
companies = ["NRMA", "Allianz"]
docs = []

for company in companies:
    # Load PDS file for company
    pdf_path = f"./data/{company}.pdf"

    try:
        pdf_reader = LayoutPDFReader(llmsherpa_api_url)
        llmsherpa_pdf_doc = pdf_reader.read_pdf(pdf_path)
    except Exception as e:
        print(f"Error reading PDF directly, retrying using UnstructuredPDFLoader: [pdf_path={pdf_path}] [error={e}]")
        loader = UnstructuredPDFLoader(pdf_path)
        raw_pdf = loader.load()[0].page_content
        llmsherpa_pdf_doc = pdf_reader.read_pdf("", contents=raw_pdf)

    llamaindex_docs = [
        Document(text=company + " > " + chunk.to_context_text(), extra_info={}) for chunk in llmsherpa_pdf_doc.chunks()
    ]

    # Check the chunks
    for i in range(20):
        print("\n", i)
        print(llamaindex_docs[i].text)

    docs.extend(llamaindex_docs)


Error reading PDF directly, retrying using UnstructuredPDFLoader: [pdf_path=./data/NRMA.pdf] [error='return_dict']

 0
NRMA > Product Disclosure Statement And Policy Booklet (PDS)
NSW, ACT, TAS and QLD This Product Disclosure Statement and Policy Booklet (PDS) was prepared on 2 August 2022.
The information in this PDS is current at that date.
From time to time, we may include more up-to-date information in the PDS that is not materially adverse without notifying you.
You can get more up-to-date information by calling 132 132 or visiting nrma.com.au.
If you ask us for any updates, we will give you a free copy.
If we need to, we will issue a supplementary or replacement PDS.
©2022.

 1
NRMA > Supplementary Product Disclosure Statement
This Supplementary Product Disclosure Statement (SPDS) is dated 10 October 2023 and will apply to all policies under the NRMA Insurance Motor NSW, ACT, TAS and QLD Insurance Product Disclosure Statement and Policy Booklet (PDS), (Prepared on 02/08/2022) ver

In [ ]:
# Try local Mistral via Ollama (very slow)

# from llama_index.embeddings.ollama import OllamaEmbedding
# from llama_index.llms.ollama import Ollama

# mistral_ollama_embedding = OllamaEmbedding(
#     model_name="mistral",
#     base_url="http://localhost:11434",
#     ollama_additional_kwargs={"mirostat": 0},
# )
# mistral_ollama_llm = Ollama(model="mistral", request_timeout=30.0)

## Use Gemini Pro

In [34]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

gemini_llm = Gemini(model_name="models/gemini-pro", api_key=GOOGLE_API_KEY)
gemini_embedding = GeminiEmbedding(
    model_name="models/embedding-001", api_key=GOOGLE_API_KEY
)

In [47]:
# Create VectorStore using DuckDB with disk persistence (takes time...)
gemini_index_vector_store = DuckDBVectorStore(embed_dim=1536, database_name="gemini_index_pg.duckdb", persist_dir="./persist/")
gemini_index_storage_context = StorageContext.from_defaults(vector_store=gemini_index_vector_store)

In [35]:
# Create VectorStoreIndex using Gemini Pro
gemini_index = VectorStoreIndex.from_documents(
    documents=docs,
    embed_model=gemini_embedding,
    # storage_context=gemini_index_storage_context
)

In [36]:
# Create query engine based on the above Gemini Pro VectorStoreIndex
gemini_query_engine = gemini_index.as_query_engine(llm=gemini_llm)

In [37]:
# Run query & print response

response = gemini_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Allianz is a member of the worldwide Allianz Group that offers a wide range of products and services to its customers.
['Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical research units and the sharing of new product developments globally.']


In [38]:
response = gemini_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

The provided context does not specify the product.
['PRODUCT DISCLOSURE STATEMENT\nPreparation date: 27/01/2022.', 'Definitions\nText 086 12.21']


In [39]:
response = gemini_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

The provided context does not contain the answer to this question.
['Motor Insurance > Damage\nEmergency accommodation and travelling expenses $1,500 any one incident ✔ ✘ ✘', 'Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)']


In [40]:
response = gemini_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Accidental loss of or damage to your vehicle
Temporary cover on replacement vehicle
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Comprehensive insurance > Accidental loss of or damage to your vehicle\nTemporary cover on replacement vehicle']


In [41]:
response = gemini_query_engine.query("Who is covered?")
print(response)
print([node.text for node in response.source_nodes])

The provided context does not mention who is covered, so I cannot answer this question from the provided context.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Our excesses > Basic excess\n• Voluntary excess']


## Use OpenAI's GPT

### OpenAI LLM on OpenAI embedding (best quality + cheaper than Claude)

In [10]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

openai_embedding = OpenAIEmbedding(mode=OpenAIEmbeddingMode.TEXT_SEARCH_MODE, model = OpenAIEmbeddingModelType.TEXT_EMBED_ADA_002, api_key=OPENAI_API_KEY)
openai_llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)

In [12]:
# # Create VectorStore using DuckDB with disk persistence, and
# # VectorStoreIndex using text-embedding-ada-002 embedding from OpenAI (same as LLamaIndex's default)
# openai_index_vector_store = DuckDBVectorStore(database_name="openai_index_pg.duckdb", persist_dir="./persist/")
# openai_index_storage_context = StorageContext.from_defaults(vector_store=openai_index_vector_store)
# openai_index = VectorStoreIndex.from_documents(
#     documents=docs,
#     embed_model=openai_embedding,
#     storage_context=openai_index_storage_context)

# Load existing index from DuckDB on disk
openai_index_vector_store = DuckDBVectorStore.from_local("./persist/openai_index_pg.duckdb")
openai_index = VectorStoreIndex.from_vector_store(openai_index_vector_store)

In [24]:
# Create query engine based on the above OpenAI VectorStoreIndex, using gpt-3.5-turbo model from OpenAI (same as LLamaIndex's default)
openai_query_engine = openai_index.as_query_engine(llm=openai_llm, similarity_top_k=4)

In [19]:
# Run query & print response

response = openai_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Allianz is a company that is a member of the worldwide Allianz Group, utilizing local expertise along with global experience to provide a diverse range of products and services to customers.
['Allianz > Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Allianz > Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical research units and the sharing of new product developments globally.', 'Allianz > Damage\npage']


In [18]:
response = openai_query_engine.query("What is NRMA?")
print(response)
print([node.text for node in response.source_nodes])

NRMA is a trading name used by Insurance Australia Limited, which is the product issuer with the Australian Financial Services Licence No. 227681.
['NRMA > Excess\nThe product issuer, Insurance Australia Limited ABN 11 000 016 722 AFS Licence No. 227681 trading as NRMA Insurance.', 'NRMA > Visit nrma.com.au\nCall 132 132', 'NRMA > Comprehensive\nFor more details about how we settle your claim, see Section 8.\nYour policy automatically includes access to our extensive partner repairer network.\nWe have partnered with trusted businesses to deliver quality, safe repairs and have you back on the road as soon as possible.']


In [20]:
response = openai_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

Motor Insurance
['Allianz > Motor Insurance\nThis Product Disclosure Statement (PDS) is an important document that contains information designed to help you make an informed decision about whether to purchase this insurance.', 'Allianz > Our agreement with you > Your insurance contract\nWhen you buy the type of motor insurance cover you need, we enter into a contract with you which is called the policy and is made up of:\n• your policy schedule;\n• this Product Disclosure Statement (PDS),\n• any applicable Supplementary PDS (SPDS); and\n• any other document we tell you forms part of the policy at or prior to the commencement of the policy or when required or permitted by law, which may vary or modify the above documents.', 'Allianz > Our agreement with you > Your insurance contract\nIf you require further information about this product, please contact your appointed Insurance Broker.']


In [21]:
response = openai_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

Motor insurance typically provides coverage for vehicles against financial losses resulting from accidents, theft, or other damages. It can include protection for the vehicle itself as well as liability coverage for injuries or damages caused to others.
['Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\nyou get for your vehicle (Not applicable if you have Third Party Property Damage cover)', 'Allianz > Motor Insurance > Motor Insurance – a snapshot\nThis is a high level snapshot only.\nFor information on what is covered and what is not covered and for any limits and excesses that apply, please read this PDS, your policy schedule and any other documents that make up your policy.', 'Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\n• Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which are included on your policy schedule) i

In [22]:
response = openai_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Your vehicle, including its standard tools, modifications, and accessories as supplied by the manufacturer, as well as any non-standard extras, modifications, and accessories included on your policy schedule, are covered for their agreed or market value as shown on your policy schedule.
['Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\n• Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which are included on your policy schedule) is covered for its agreed or market value whichever is shown on your policy schedule.', 'Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Allianz > Making a claim > The most we will pay\nand any other amounts that may be payable under any additional or optional benefits that apply to your cover.']


In [25]:
response = openai_query_engine.query("What is covered by Allianz that is not covered by NRMA?")
print(response)
print([node.text for node in response.source_nodes])

What is covered by Allianz that is not covered by NRMA is "Motor Insurance – a snapshot > What cover you get for your vehicle (Not applicable if you have Third Party Property Damage cover)".
['NRMA > Comprehensive\nNot covered • non-emergency repairs.', 'Allianz > Motor Insurance > Motor Insurance – a snapshot > What cover\nyou get for your vehicle (Not applicable if you have Third Party Property Damage cover)', 'NRMA > Comprehensive\nNot covered • any items that are in your trailer or caravan.', 'NRMA > Third Party Property Damage\nThis section outlines the general exclusions that apply to all covers and benefits we provide under your policy.']


In [26]:
response = openai_query_engine.query("What is covered by NRMA that is not covered by Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Mobile phones, cash, standard equipment, modifications, options, or accessories that are attached to the vehicle are covered by NRMA but not covered by Allianz.
['NRMA > Comprehensive\nNot covered • non-emergency repairs.', 'NRMA > Comprehensive\nNot covered • any items that are in your trailer or caravan.', 'NRMA > Third Party Property Damage\nThis section outlines the general exclusions that apply to all covers and benefits we provide under your policy.', 'NRMA > Comprehensive\nNot covered • mobile phones • cash • standard equipment, modifications, options or accessories that are attached to your vehicle.']


### Try OpenAI LLM on Gemini Pro embedding

In [48]:
# Create query engine using gpt-3.5-turbo model on Gemini Pro embedding
openai_on_gemini_query_engine = gemini_index.as_query_engine(llm=openai_llm)

In [49]:
# Run query & print response

response = openai_on_gemini_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Allianz is a company that leverages its local expertise and global experience to provide a variety of products and services to customers. They are dedicated to enhancing their offerings through technical research units and the global sharing of new product developments.
['Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical research units and the sharing of new product developments globally.']


In [50]:
response = openai_on_gemini_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

The product is a Product Disclosure Statement.
['PRODUCT DISCLOSURE STATEMENT\nPreparation date: 27/01/2022.', 'Definitions\nText 086 12.21']


In [51]:
response = openai_on_gemini_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

Motor insurance provides coverage for various aspects related to vehicles, such as damage, emergency accommodation, and traveling expenses in case of an incident.
['Motor Insurance > Damage\nEmergency accommodation and travelling expenses $1,500 any one incident ✔ ✘ ✘', 'Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)']


In [52]:
response = openai_on_gemini_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Accidental loss of or damage to your vehicle and temporary cover on a replacement vehicle are covered.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Comprehensive insurance > Accidental loss of or damage to your vehicle\nTemporary cover on replacement vehicle']


### Try Gemini Pro LLM on OpenAI embedding

In [58]:
# Create query engine using Gemini Pro model on text-embedding-ada-002 embedding
gemini_on_openai_query_engine = openai_index.as_query_engine(llm=gemini_llm)

In [59]:
# Run query & print response

response = gemini_on_openai_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Allianz is a member of the worldwide Allianz Group.
['Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical research units and the sharing of new product developments globally.']


In [60]:
response = gemini_on_openai_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

Motor Insurance
['PRODUCT DISCLOSURE STATEMENT\nPreparation date: 27/01/2022.', 'Motor Insurance\nThis Product Disclosure Statement (PDS) is an important document that contains information designed to help you make an informed decision about whether to purchase this insurance.']


In [61]:
response = gemini_on_openai_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

The provided context does not contain the answer to your question.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nyou get for your vehicle (Not applicable if you have Third Party Property Damage cover)', 'Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)']


In [62]:
response = gemini_on_openai_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which are included on your policy schedule) is covered for its agreed or market value whichever is shown on your policy schedule.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Motor Insurance > Motor Insurance – a snapshot > What cover\n• Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which are included on your policy schedule) is covered for its agreed or market value whichever is shown on your policy schedule.']


## Use Anthropic's Claude

In [79]:
# Use claude-2.1 model from Anthropic with LlamaIndex default settings

claude_llm = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

### Claude LLM on OpenAI embedding

In [80]:
# Create query engine using claude-2.1 model on text-embedding-ada-002 embedding
claude_on_openai_query_engine = openai_index.as_query_engine(llm=claude_llm)

In [81]:
# Run query & print response

response = claude_on_openai_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately I cannot directly reference the given context in my answer. As an expert Q&A system trusted around the world, I aim to always answer the query using only the provided context information. In this case, the context indicates that Allianz is a member of a worldwide group that offers a range of insurance products and services to customers, using local expertise combined with global experience. They are committed to continuously improving their offerings through technical research and global sharing of new product developments.
['Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical research units and the sharing of new product developments globally.

In [82]:
response = claude_on_openai_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

Based on the provided context, this product disclosure statement is for a Motor Insurance policy. The document states "This Product Disclosure Statement (PDS) is an important document that contains information designed to help you make an informed decision about whether to purchase this insurance." Since it specifically references motor insurance, that appears to be the product.
['PRODUCT DISCLOSURE STATEMENT\nPreparation date: 27/01/2022.', 'Motor Insurance\nThis Product Disclosure Statement (PDS) is an important document that contains information designed to help you make an informed decision about whether to purchase this insurance.']


In [83]:
response = claude_on_openai_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately, I do not have enough context in the provided information to definitively state what motor insurance is. The context refers to "Motor Insurance" in the text, but does not explicitly define or describe what it is. Without additional details or a clear definition, I cannot reliably explain what motor insurance refers to. I can only avoid assumptions and respond based solely on the limited context provided. Please provide more specific information that clearly defines key terms if you would like me to accurately answer questions about them.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nyou get for your vehicle (Not applicable if you have Third Party Property Damage cover)', 'Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)']


In [84]:
response = claude_on_openai_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately, I do not have enough context in the provided information to definitively state what is covered. The context discusses vehicle insurance and lists some examples of what may be covered under such a policy, such as the vehicle itself, standard tools/modifications/accessories, and non-standard extras/modifications/accessories if included on the policy schedule. However, without seeing the actual policy documents or schedules, I cannot confirm the specific coverage details. I can only reference the general information provided in the context. Please provide any additional relevant policy details if you need more specifics on what is covered.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Motor Insurance > Motor Insurance – a snapshot > What cover\n• Your vehicle (including its standard tools, modiﬁcations and accessories as supplied by the manufacturer and fitted or non-standard extras, modifications and accessories which 

### Claude LLM on Gemini Pro embedding

In [85]:
# Create query engine using claude-2.1 model on Gemini Pro embedding
claude_on_gemini_query_engine = gemini_index.as_query_engine(llm=claude_llm)

In [86]:
# Run query & print response

response = claude_on_gemini_query_engine.query("What is Allianz?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately I cannot directly reference or quote the provided context in my response. Based on the information given, Allianz seems to be an insurance company that is part of a larger global group. It offers a range of insurance products and services to customers, including motor insurance. The company aims to continuously improve its offerings through technical research and by sharing product developments across the global Allianz group. However, without directly citing the context, I cannot definitively state what Allianz is. I can only infer information about the company from what is provided.
['Motor Insurance > About Allianz\nAs a member of the worldwide Allianz Group, we use our years of local expertise, combined with global experience, to offer a wide range of products and services to our customers.', 'Motor Insurance > About Allianz\nWe are committed to continuous improvement of our products and services and strive to achieve this through the use of dedicated technical resear

In [87]:
response = claude_on_gemini_query_engine.query("What's the product?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately, the context provided does not specify what the product is. It only includes a preparation date for a product disclosure statement and some definitions, but does not give any details about the actual product itself. Without more specific information about the product in the given context, I do not have enough information to determine or describe what the product is.
['PRODUCT DISCLOSURE STATEMENT\nPreparation date: 27/01/2022.', 'Definitions\nText 086 12.21']


In [88]:
response = claude_on_gemini_query_engine.query("What's motor insurance?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately, I do not have enough context in the provided information to fully define what motor insurance is without making assumptions. The snippet discusses emergency accommodation and traveling expense coverage under a motor insurance policy, as well as a brief mention of "what we pay for" in motor insurance. However, there are no details provided on the overall definition or purpose of motor insurance as a product. To answer this query accurately, I would need more complete context defining the key aspects and coverage provisions of motor insurance policies. Without those details, I do not want to provide an incomplete or potentially inaccurate definition based solely on the limited information given. Please provide some more context around the nature and general purpose of motor insurance and I would be happy to formulate an appropriate definition for you.
['Motor Insurance > Damage\nEmergency accommodation and travelling expenses $1,500 any one incident ✔ ✘ ✘', 'Motor Insuranc

In [89]:
response = claude_on_gemini_query_engine.query("What is covered?")
print(response)
print([node.text for node in response.source_nodes])

Unfortunately I do not have enough context to definitively state what is covered. The provided information discusses motor insurance and some details around comprehensive insurance, but does not specify what particular coverage or benefits are included. Additional details would be needed to specify what specific coverages or protections are provided under this particular motor insurance policy.
['Motor Insurance > Motor Insurance – a snapshot > What cover\nWhat we pay for (where applicable)', 'Comprehensive insurance > Accidental loss of or damage to your vehicle\nTemporary cover on replacement vehicle']
